In [6]:
import pandas as pd
import glob

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Define the path to the directory containing the Excel files
path = r'source files'

# Get a list of file names in the directory that end with .xlsx
file_list = glob.glob(path + "/*.xlsx")

# Create an empty list to hold the DataFrames
df_list = []

# Loop over the file names and read each file into a DataFrame
for file in file_list:
    df = pd.read_excel(file, header=1)

    # Clean and format the data according to the specified rules
    df = df.dropna(axis=1, how='all')
    df = df[~df['o'].isin([0.1, 0.2, 0.3])]

    df['units'] = pd.to_numeric(df['units'], errors='coerce')
    df['comuMerged'] = df['comu'].fillna(df['comu_2'])

    # Drop rows with NaN values in the "comuMerged" column
    df = df.dropna(subset=['comuMerged'])

    # Convert the "comuMerged" column to a float, round it to remove decimals, then convert to an integer, and finally to a string
    df['comuMerged'] = df['comuMerged'].astype(float).round(0).astype(int).astype(str)


    # Add a leading zero to the "comuMerged" column if it has only 3 digits
    df['comuMerged'] = df['comuMerged'].str.zfill(4)

    # Combine the "area" and "phone" columns
    df['combinedPhone'] = df['area'].astype(str) + ' ' + df['phone'].astype(str)

    # Select the desired columns
    cleaned_df = df[['comuMerged', 'development / aka', 'units', 'type', 'tenure', 'agent', 'agent address', 'website', 'source', 'zip', 'combinedPhone']]

    df_list.append(cleaned_df)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(df_list)

# Save the resulting DataFrame to a CSV file
combined_df.to_csv('Municipal-level-housing-data.csv', index=False)

# Group by 'comuMerged' and calculate the total 'units' for each group
total_units_per_comu = combined_df.groupby('comuMerged')['units'].sum().reset_index()

# Convert the 'units' column to integers
total_units_per_comu['units'] = total_units_per_comu['units'].astype(int)

# Read the 'simplified data 7.xlsx' file
simplified_data = pd.read_excel('jacobson files/simplified data 7.xlsx')

# Rename the 'Muni code' column to 'comuMerged', convert it to an integer, and then to a string with leading zeros for 3-digit values
simplified_data = simplified_data.rename(columns={'Muni Code': 'comuMerged'})
simplified_data['comuMerged'] = simplified_data['comuMerged'].astype(int).astype(str).str.zfill(4)

# Merge the "total_units_per_comu" DataFrame with the "simplified_data" DataFrame
merged_df = total_units_per_comu.merge(
    simplified_data[['comuMerged', 'Prior Round Obligations (1987-1999)', 'Present Need (2015)', 'Gap Present Need (2015)', 'Prospective Need (2015-2025)', 'Total', 'Prospective + Gap Present']],
    on='comuMerged',
    how='right'
)

# Fill cells without a value with "0", remove ".0" from the end of all non-zero float values, and save the updated DataFrame to the "Total_Units_per_Comu.csv" file
merged_df = merged_df.fillna(0)
merged_df = merged_df.applymap(lambda x: str(x).rstrip('.0') if isinstance(x, float) and x != 0 else '0' if x == 0 else x)
merged_df.to_csv('Total_Units_per_Comu.csv', index=False)

# Display the top 50 and bottom 50 rows of the merged DataFrame
display(pd.concat([merged_df.head(50), merged_df.tail(50)]))



,comuMerged,units,Prior Round Obligations (1987-1999),Present Need (2015),Gap Present Need (2015),Prospective Need (2015-2025),Total,Prospective + Gap Present
0,0101,403,144,51,40,15,250,55
1,0102,4813,2458,828,0,0,3286,0
2,0103,13,124,38,28,175,365,203
3,0104,238,41,7,10,15,73,25
4,0105,0,19,67,22,11,119,33
5,0106,0,13,1,8,14,36,22
6,0107,327,42,52,16,0,110,16
7,0108,472,763,101,289,251,1404,540
8,0109,0,21,0,15,10,46,25
9,0110,0,20,1,15,10,46,25
